In [1]:
# Set number of threads to use
import os
nthreads = 12
os.environ["MKL_NUM_THREADS"] = str(nthreads)
os.environ["NUMEXPR_NUM_THREADS"] = str(nthreads)
os.environ["OMP_NUM_THREADS"] = str(nthreads)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1,'../src/')

In [3]:
from supercellHelpers import *
from buildGRNHelpers import *
from MergeNetworksHelpers import *

In [4]:
#adata = sc.read('../data/microglia.h5ad')

In [5]:
#adata

In [6]:
"""adata_merged = pipeline(adata,
                       ngenes=2000,
                       npcs=20,
                       ncell=500,
                       verbose=True)"""

'adata_merged = pipeline(adata,\n                       ngenes=2000,\n                       npcs=20,\n                       ncell=500,\n                       verbose=True)'

In [7]:
adata_merged = sc.read('../temp_data/merged_adata.h5ad')

# limit genes to 2000 random genes
# this is to speed up computation for the example
np.random.seed(0)
adata_merged = adata_merged[:,np.random.choice(np.arange(adata_merged.shape[1]),
                                       2000,
                                       replace=False)]

In [8]:
adata_merged

View of AnnData object with n_obs × n_vars = 503 × 2000
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [9]:
all_edges = []
for i in range(10):
    print(i)
    adata_saved = adata_merged.copy()
    
    # -1 all genes
    # 100 neighbors for each gene
    # 10 pcs
    # 0.7 subsample per run
    scing = grnBuilder(adata_merged, -1, 100, 10,0.7,
                      'test','test',12,int(2e9),True)
    scing.subsample_cells()

    scing.filter_genes()
    scing.filter_gene_connectivities()
    scing.build_grn()
    
    all_edges.append(scing.edges)

0
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
1
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
2
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
3
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
4
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
5
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
6
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
7
building local cluster
Loading data into memory...
Building dask graph...
Computing dask graph...
Closing client...
8
building local cluster
Loading data into memory...
Bui

In [10]:
import pickle as pkl
with open('../temp_data/edge.pkl','wb') as f:
    pkl.dump(all_edges, f)
    #all_edges = pkl.load(f)

In [11]:
merger = NetworkMerger(adata_merged,
                    all_edges,
                       0.2,
                    'test',
                    'test',
                    12,
                    int(2e9),
                    True)

In [12]:
merger.preprocess_network_files()
merger.remove_reversed_edges()
merger.remove_cycles()
merger.get_triads()
merger.remove_redundant_edges()

Preprocessing in network files...
Summarizing networks...
Removing bidirectional edges...
Removing cycles...
Getting triads to remove redundant edges...
Removing redundant edges...
Creating client...
Loading data...
Building dask graph...
Closing client...
Removing edges...


In [17]:
merger.edge_df.sort_values(by='importance',
                          ascending=False)

,source,target,importance
319,AC011444.1,AC099343.3,378.428711
27,AC096711.2,ARHGEF16,376.240692
394,MT-ATP6,MT-ND2,363.492862
4623,C18orf21,PSMB9,361.331336
11269,MT-ND2,MT-ATP6,330.889865
...,...,...,...
9540,TRPT1,SRRM3,11.391193
7255,SLITRK2,RGCC,11.365703
3480,FAM177A1,FANCL,11.279903
9851,CARHSP1,MT-ND2,10.889385
